# TV Script Generation
In this project, you'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  You'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
The data is already provided for you.  You'll be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.251908396946565
Number of lines: 4258
Average number of words in each line: 11.50164396430249

The sentences 0 to 10:

Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.



## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

### This was my original method. The next cell used dictionary comprehension and it is more concise. I kept this because I did do some research to get it to work and it highlights some differences between python 2 and 3. This can be refered to later in the future.

In [3]:
# import numpy as np
# import problem_unittests as tests
# from collections import Counter

# # Inverting a Dictionary
# # https://www.safaribooksonline.com/library/view/python-cookbook-2nd/0596007973/ch04s15.html
# def invert_dict(d):
    
#     # NOTE: .iteritems is deprecated in python 3, use .items() instead
#     # https://stackoverflow.com/questions/30418481/error-dict-object-has-no-attribute-iteritems-when-trying-to-use-networkx
#     # https://wiki.python.org/moin/Python3.0
#     #return dict([ (v, k) for k, v in d.iteritems( ) ])
#     return dict([ (v, k) for k, v in d.items( ) ])

# # # Inverting a Dictionary
# # # Faster for larger dictionaries
# # # https://www.safaribooksonline.com/library/view/python-cookbook-2nd/0596007973/ch04s15.html
# # from itertools import izip
# # def invert_dict_fast(d):
# #     return dict(izip(d.itervalues( ), d.iterkeys( )))

# # Inverting a Dictionary
# # Faster for larger dictionaries
# # https://www.safaribooksonline.com/library/view/python-cookbook-2nd/0596007973/ch04s15.html
# #
# # NOTE: In python 3, there are some changes.
# #       - You don't need itertools izip, you can just use zip 
# #         http://www.diveintopython3.net/porting-code-to-python-3-with-2to3.html
# #       - itervalues and iterkeys are deprecated dictionary methods, use dict.keys() and dict.values()
# #         https://wiki.python.org/moin/Python3.0
# def invert_dict_fast(d):
#     return dict(zip(d.values( ), d.keys( )))


# def create_lookup_tables(text):
#     """
#     Create lookup tables for vocabulary
#     :param text: The text of tv scripts split into words
#     :return: A tuple of dicts (vocab_to_int, int_to_vocab)
#     """
#     # TODO: Implement Function
#     counts = Counter(text)
#     vocab = sorted(counts, key=counts.get, reverse=True)
    
#     # Dictionary comprehension.
#     #
#     # https://stackoverflow.com/questions/22171558/what-does-enumerate-mean
#     # The enumerate() function adds a counter to an iterable.
#     # So for each element in cursor, a tuple is produced with (counter, element); 
#     # the for loop binds that to row_number and row, respectively.
#     # By default, enumerate() starts counting at 0 but if you give it a second integer 
#     # argument, it'll start from that number instead:
#     vocab_to_int = {text: ii for ii, text in enumerate(vocab, 1)}
    
#     # Create the inverted dictionary
#     #int_to_vocab = invert_dict(vocab_to_int)
#     int_to_vocab = invert_dict_fast(vocab_to_int)

    
#     return vocab_to_int, int_to_vocab


# """
# DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
# """
# tests.test_create_lookup_tables(create_lookup_tables)

### Using dictionary comprehension is more concise.

In [4]:
import numpy as np
import problem_unittests as tests
from collections import Counter

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
    counts = Counter(text)
    vocab = sorted(counts, key=counts.get, reverse=True)
    vocab_to_int = {word:i for i, word in enumerate(vocab)}
    int_to_vocab = {i:word for i, word in enumerate(vocab)}
    return vocab_to_int, int_to_vocab


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [5]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    dict_punctuation_tokens = { '.' : '||Period||',
                                ',' : '||Comma||',
                                '"' : '||Quotation_Mark||',
                                ';' : '||Semicolon||',
                                '!' : '||Exclamation_mark||',
                                '?' : '||Question_mark||',
                                '(' : '||Left_Parentheses||',
                                ')' : '||Right_Parentheses||',
                                '--' : '||Dash||',
                                '\n' : '||Return||' }    
    return dict_punctuation_tokens

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [7]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [8]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.0.1
Default GPU Device: /gpu:0


### Input
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following tuple `(Input, Targets, LearningRate)`

In [9]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    # TODO: Implement Function
    Input = tf.placeholder(tf.int32, shape = [None, None], name='input')
    Targets = tf.placeholder(tf.int32,  shape = [None, None], name='Targets')
    LearningRate = tf.placeholder(tf.float32, shape = None, name='LearningRate')
    
    return Input, Targets, LearningRate


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)

Tests Passed


### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [18]:
def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    # TODO: Implement Function
    
    # NOTE: It seems like setting the number of layers matter when training
    #       as well as dropout probability. It would be nice if these 
    #       "hyper-parameters" were consolidated into one place so I don't have to jump around
    #        the notebook.
    #num_layers = 10
    num_layers = 1
    keep_prob = 0.5
    
    # Your basic LSTM cell
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
        
    # Add dropout
    drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
    
    # Stack up multiple LSTM layers, for deep learning
    #
    # https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell
    # https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell
    #
    # Note:
    # RNN cell composed sequentially of multiple simple cells.
    # Even if it is just one BasicLSTMCells you have to make it iterable by
    # puttint it in a dictionary.
    #
    # Note: Put 3 layers because the lecture stated that 3 layers improves 2 but
    #       no real difference beyound 3 unless dealing with convolution network.
    #       RNN is similar to a traditional feed-forward network, so I will assume
    #       3 layers is a good starting point.
    Cell = tf.contrib.rnn.MultiRNNCell([drop] * num_layers)
    
    # Getting an initial state of all zeros
    InitialState = Cell.zero_state(batch_size, tf.float32)
    
    # https://www.tensorflow.org/api_docs/python/tf/identity
    InitialState = tf.identity(InitialState, name='initial_state')
    
    return Cell, InitialState


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

In [11]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    # TODO: Implement Function
    
    # Examples of embedding:
    # https://github.com/udacity/deep-learning/blob/master/embeddings/Skip-Grams-Solution.ipynb
    # https://github.com/udacity/deep-learning/blob/master/sentiment-rnn/Sentiment_RNN_Solution.ipynb
    embedding = tf.Variable(tf.random_uniform((vocab_size, embed_dim), -1, 1))
    embed = tf.nn.embedding_lookup(embedding, input_data)
    return embed


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)

Tests Passed


### Build RNN
You created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [12]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    # TODO: Implement Function
    
    # https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn
    Outputs, FinalState = tf.nn.dynamic_rnn( cell,
                                             inputs,
                                             dtype = tf.float32 )
    
    # https://www.tensorflow.org/api_docs/python/tf/identity
    FinalState = tf.identity(FinalState, name='final_state')
    
    return Outputs, FinalState


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

### NOTE: rnn_size does not seem to be used.

In [13]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    # TODO: Implement Function
    
    # Apply embedding to input_data using your get_embed(input_data, vocab_size, embed_dim) function.
    #embed = get_embed(input_data, vocab_size, embed_dim)
    embed = get_embed(input_data, vocab_size, embed_dim)
    
    # Build RNN using cell and your build_rnn(cell, inputs) function.
    Outputs, FinalState = build_rnn(cell, embed)

    # Apply a fully connected layer with a linear activation and vocab_size as the number of outputs.
    # https://www.tensorflow.org/api_docs/python/tf/contrib/layers/fully_connected
    #
    # Note: To have linear activation, pass None to activation_fn
    Logits  = tf.contrib.layers.fully_connected( Outputs,
                                                 vocab_size,
                                                 activation_fn=None,
                                                 weights_initializer = tf.truncated_normal_initializer(stddev=0.1),
                                                 biases_initializer=tf.zeros_initializer())
    
    
    return Logits, FinalState


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

Tests Passed


### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If you can't fill the last batch with enough data, drop the last batch.

For exmple, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2], [ 7  8], [13 14]]
    # Batch of targets
    [[ 2  3], [ 8  9], [14 15]]
  ]

  # Second Batch
  [
    # Batch of Input
    [[ 3  4], [ 9 10], [15 16]]
    # Batch of targets
    [[ 4  5], [10 11], [16 17]]
  ]

  # Third Batch
  [
    # Batch of Input
    [[ 5  6], [11 12], [17 18]]
    # Batch of targets
    [[ 6  7], [12 13], [18  1]]
  ]
]
```

Notice that the last target value in the last batch is the first input value of the first batch. In this case, `1`. This is a common technique used when creating sequence batches, although it is rather unintuitive.

In [14]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    # TODO: Implement Function
    chars_in_a_batch = batch_size * seq_length
    
    # NOTE: In python 3, double slashes // means interger division.
    # https://stackoverflow.com/questions/1535596/what-is-the-reason-for-having-in-python
    num_batches = len(int_text)//(chars_in_a_batch)
    
    # Use slicing to select the range.
    # NOTE: The target is offset by 1 from the input. That is because it is a recurrent network,
    #       the previous input is needed to predict the next output. The example supplied in the 
    #       markdown cell above demonstrates this phenomenon.
    input_data = np.array(int_text[ : (num_batches * chars_in_a_batch)])
    target_data = np.array(int_text[1 : (num_batches * chars_in_a_batch)+1])
    
    # The last target value in the last batch is the first input value of the first batch.
    # Note: I don't fully understand why this works. It was mentioned in the lession it is
    #       not understood why this works either. I am doing it because it was mentioned in
    #       the lecture and in the instruction.
    target_data[-1] = input_data[0] 
    
    # Reshape
    # https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.reshape.html
    # 
    # Note:
    # One shape dimension can be -1. In this case, the value is inferred from the length of the array and remaining dimensions.
    inputs = input_data.reshape(batch_size, -1)
    targets = target_data.reshape(batch_size, -1)

    # Numpy Split
    # https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.split.html
    #
    # Note: This split an array into multiple sub-arrays.
    # NOTE: Use axis 1.
    inputs = np.split(inputs, num_batches, 1)
    targets = np.split(targets, num_batches, 1)
    
    # Outputting to format (number of batches, 2, batch size, sequence length)
    batches = np.array(list(zip(inputs, targets)))

    return batches


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_batches(get_batches)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `embed_dim` to the size of the embedding.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

### NOTE: These were my original hyperparameter settings that trained on. I left them because I let it train for about 5 hours on GPU-enabled tensorflow and it hovered around 2.5 and 3.0 loss value before I stopped it. I stopped it around 283 epochs out of 300. It did not look like it was going to get better. I keep it around for reference in the future.

In [15]:
# # Number of Epochs
# num_epochs = 300
# # Batch Size
# batch_size = 32
# # RNN Size
# rnn_size = 300
# # Embedding Dimension Size
# #embed_dim = 200
# embed_dim = 300
# # Sequence Length
# seq_length = 10
# # Learning Rate
# learning_rate = 0.01
# # Show stats for every n number of batches
# show_every_n_batches = 10

# """
# DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
# """
# save_dir = './save'

### These are the hyperparameter settings I used that worked. It actually got the desired target loss value of less than 1 in about 10 minutes on a GPU enabled tensorflow (as opposed 5 hours) at around epoch 12. I left it for a little over an hour with 100 epochs. The loss seemed to hover at about 0.6 from epoch 22 and onward. With these settings and the hardward used to train the model, leaving it about 20 to 30 epochs for about 30 min would probably best optimize this configuration. The rest of epochs are most likely a waste of time. Of course, you can't know that when discovering the settings.

In [20]:
# Number of Epochs
num_epochs = 100
# Batch Size
batch_size = 256
# RNN Size
rnn_size = 1024
# Embedding Dimension Size
#embed_dim = 200
embed_dim = 1024
# Sequence Length
seq_length = 15
# Learning Rate
learning_rate = 0.01
# Show stats for every n number of batches
show_every_n_batches = 20

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

### Build the Graph
Build the graph using the neural network you implemented.

In [21]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forums](https://discussions.udacity.com/) to see if anyone is having the same problem.

### As mentioned above, this was my 5 hour GPU-enabled tensorflow settings. I kept it for reference.

In [17]:
# """
# DON'T MODIFY ANYTHING IN THIS CELL
# """
# batches = get_batches(int_text, batch_size, seq_length)

# with tf.Session(graph=train_graph) as sess:
#     sess.run(tf.global_variables_initializer())

#     for epoch_i in range(num_epochs):
#         state = sess.run(initial_state, {input_text: batches[0][0]})

#         for batch_i, (x, y) in enumerate(batches):
#             feed = {
#                 input_text: x,
#                 targets: y,
#                 initial_state: state,
#                 lr: learning_rate}
#             train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

#             # Show every <show_every_n_batches> batches
#             if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
#                 print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
#                     epoch_i,
#                     batch_i,
#                     len(batches),
#                     train_loss))

#     # Save Model
#     saver = tf.train.Saver()
#     saver.save(sess, save_dir)
#     print('Model Trained and Saved')

Epoch   0 Batch    0/215   train_loss = 8.821
Epoch   0 Batch   10/215   train_loss = 6.674
Epoch   0 Batch   20/215   train_loss = 6.200
Epoch   0 Batch   30/215   train_loss = 5.878
Epoch   0 Batch   40/215   train_loss = 6.115
Epoch   0 Batch   50/215   train_loss = 5.896
Epoch   0 Batch   60/215   train_loss = 5.906
Epoch   0 Batch   70/215   train_loss = 5.672
Epoch   0 Batch   80/215   train_loss = 5.504
Epoch   0 Batch   90/215   train_loss = 5.512
Epoch   0 Batch  100/215   train_loss = 5.502
Epoch   0 Batch  110/215   train_loss = 5.424
Epoch   0 Batch  120/215   train_loss = 5.449
Epoch   0 Batch  130/215   train_loss = 5.825
Epoch   0 Batch  140/215   train_loss = 5.918
Epoch   0 Batch  150/215   train_loss = 5.461
Epoch   0 Batch  160/215   train_loss = 5.382
Epoch   0 Batch  170/215   train_loss = 5.493
Epoch   0 Batch  180/215   train_loss = 5.547
Epoch   0 Batch  190/215   train_loss = 5.433
Epoch   0 Batch  200/215   train_loss = 5.351
Epoch   0 Batch  210/215   train_l

KeyboardInterrupt: 

### These are the hyperparameter settings that got the result I was looking for, loss of < 1.

In [22]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/17   train_loss = 8.917
Epoch   1 Batch    3/17   train_loss = 5.055
Epoch   2 Batch    6/17   train_loss = 4.128
Epoch   3 Batch    9/17   train_loss = 3.341
Epoch   4 Batch   12/17   train_loss = 2.658
Epoch   5 Batch   15/17   train_loss = 2.198
Epoch   7 Batch    1/17   train_loss = 1.812
Epoch   8 Batch    4/17   train_loss = 1.556
Epoch   9 Batch    7/17   train_loss = 1.342
Epoch  10 Batch   10/17   train_loss = 1.168
Epoch  11 Batch   13/17   train_loss = 1.069
Epoch  12 Batch   16/17   train_loss = 0.989
Epoch  14 Batch    2/17   train_loss = 0.935
Epoch  15 Batch    5/17   train_loss = 0.852
Epoch  16 Batch    8/17   train_loss = 0.862
Epoch  17 Batch   11/17   train_loss = 0.768
Epoch  18 Batch   14/17   train_loss = 0.772
Epoch  20 Batch    0/17   train_loss = 0.740
Epoch  21 Batch    3/17   train_loss = 0.726
Epoch  22 Batch    6/17   train_loss = 0.688
Epoch  23 Batch    9/17   train_loss = 0.628
Epoch  24 Batch   12/17   train_loss = 0.667
Epoch  25 

## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [23]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [24]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [27]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    # TODO: Implement Function
    
    # Get the tensors by name
    #
    # NOTE: It seems like tensorflow enumerates the variable names you give it,
    #       so I appened :0 to the variable names since there should be only one copy.
    InputTensor = loaded_graph.get_tensor_by_name('input:0')
    InitialStateTensor = loaded_graph.get_tensor_by_name('initial_state:0')
    FinalStateTensor = loaded_graph.get_tensor_by_name('final_state:0')
    ProbsTensor = loaded_graph.get_tensor_by_name('probs:0')
    
    return InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_tensors(get_tensors)

Tests Passed


### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [40]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    # TODO: Implement Function
    
#     # DEBUG
#     #
#     print("DEBUG - type(probabilities): {}".format(type(probabilities)))
#     print("DEBUG - type(int_to_vocab): {}".format(type(int_to_vocab)))
    
    # Generates a random sample from a given 1-D array
    # https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.choice.html
    pred_word = np.random.choice( list(int_to_vocab.values()),
                                  p=probabilities )
    
    return pred_word


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_pick_word(pick_word)

Tests Passed


## Generate TV Script
This will generate the TV script for you.  Set `gen_length` to the length of TV script you want to generate.

In [41]:
gen_length = 200
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'moe_szyslak'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

moe_szyslak: oh no!(four hundred nos.
carl_carlson: marge, we gotta celebrate! throw a ragin' on, and then there...
comic_book_guy: my name is walther hotenhoffer and i'm in an here.
marge_simpson:(pained) i guess so one, i'd'a put lenny on the...
homer_simpson:(getting idea) no, the pipes not smells so much about lousy.
moe_szyslak: wow, that i go with that uh, y'know they mr. the private, show me the secret guy i know named to make my rent.
carl_carlson: be for him? every day too?
moe_szyslak: 'cause i was inches of you lugs wanna get any specific harm.


moe_szyslak: get down here! the you!
barney_gumble: of course. but didn't it sounds like one of these" alone during the bonding phase.
moe_szyslak:(desperate) wait, wait, wait, wait, wait... bo's he still got to say one hundred.
marguerite: is here that the now of you krusty.



# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckly there's more data!  As we mentioned in the begging of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.
# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.